In [11]:
# Utils imports
import pandas as pd
import numpy as np
import os
import joblib
import random

#Optimization imports
import optuna as opt

#Evaluation imports
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score


In [12]:
#Model imports
from xgboost import XGBRegressor
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor

In [13]:
# Preparing data

df = pd.read_csv("../dataset/smogn_syn_data.csv")
tar_col = "LC50 [-LOG(mol/L)]"
Syn = "isSyn"
isSyn = df[Syn]
X = df.drop([tar_col,Syn], axis=1)
Y = df[tar_col]
isSyn


# PARAMS 
NUM_TRIALS = 15

In [14]:
# Creating the dict for model and trial values
# models = {
#     "XGBRegressor" : {
#         "model" : XGBRegressor,
#         "param" : {
#                 "n_estimators":  'trial.suggest_categorical("xgb_est",[4500,5000])',
#                 "learning_rate": 'trial.suggest_categorical("xgb_lr",[0.01,3e-4,0.1])',
#                 "booster" : 'trial.suggest_categorical("xgb_booster",["gbtree","gblinear","dart"])',
#                 "tree_method" : 'trial.suggest_categorical("xgb_treemethod",["gpu_hist"])',
#                 "predictor" : 'trial.suggest_categorical("xgp_predictor",["gpu_predictor"])'
#         }
#     }
# }   

In [15]:
# # main objective function for optuna 
# def train_main(X,Y,isSyn,train_fold,models):
#     train_fold = joblib.load("../exports/train_test_fold_data.z")
#     fold=0
#     train_index = train_fold[fold]["train"]
#     test_index = train_fold[fold]["test"]
#     X_main = X.iloc[train_index, :].to_numpy(dtype=np.float64)
#     Y_main = Y[train_index].to_numpy(dtype=np.float64)
#     X_test = X.iloc[test_index, :].to_numpy(dtype=np.float64)
#     Y_test = Y[test_index].to_numpy(dtype=np.float64)
#     out_data = {}
#     for model_name,model_item in models.items():
#         def objective(trial):
#             clf = model_item["model"](model_item["param"])
#             clf.fit(X_main, Y_main,
#                     eval_set = [(X_test, Y_test)]
#                     eval_metric = ["rmse"])
#             Y_pred = clf.predict(X_test)
#             error = mean_squared_error(Y_pred, Y_test, squared=False)
#             return error
#         study = opt.create_study(direction='minimize')
#         study.optimize(objective, n_trials = NUM_TRIALS)
#         best_params = study.best_params
#         trial_data = trial.get_trials()
#         clf_main= model_item["model"](**best_params)
#         clf.fit(X_main, Y_main,
#                 eval_set = [(X_test, Y_test)]
#                 eval_metric = ["rmse"])
#         Y_pred_main = clf.predict(X_test)
#         error_metrics_all = {
#             "mse_error" : mean_squared_error(Y_pred_main,Y_test),
#             "mae_error" : mean_absolute_error(Y_pred_main,Y_test),
#             "rmse_error" : mean_squared_error(Y_pred_main,Y_test, squared=False),
#             "r2_score" : r2_score(Y_pred_main,Y_test)}
        
#         out_data[model_name] = { "best_params"  : best_params,
#                                  "trial_data" : trial_data,
#                                 "error_metric_all": error_metrics_all,
                                
                                 
        
        

In [16]:
# sample model testing 
train_fold = joblib.load("../exports/train_test_fold_data.z")
tot_fold = 15
error_vals = {}
for fold in range(tot_fold):
    X_train = X.loc[train_fold[fold]["train"],:]
    Y_train = Y.loc[train_fold[fold]["train"]]

    X_eval = X.loc[train_fold[fold]["test"],:]
    Y_eval = Y.loc[train_fold[fold]["test"]]
    print(fold,"  ", X_train.shape,Y_train.shape, X_eval.shape, Y_eval.shape)
    reg = XGBRegressor(n_estimators=15000,
                       learning_rate = 3e-4,
                       booster = "gbtree",
                       tree_method = "gpu_hist",
                       predictor = "gpu_predictor")
    # X_train = X_train.to_numpy(dtype=np.float64)
    # Y_train = Y_train.to_numpy(dtype=np.float64)
    reg.fit(X_train,
            Y_train,
            eval_set = [(X_eval, Y_eval)],
            eval_metric = ["rmse"])
    Y_pred = reg.predict(X_eval)
    mse_error = mean_squared_error(Y_pred,Y_eval, squared=False)
    error_vals[fold] = mse_error
    joblib.dump(reg,f"{fold}_xgb_reg.z")
joblib.dump(error_vals,f"errors.z")

0    (1002, 6) (1002,) (72, 6) (72,)


C:\Users\avyar\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-rmse:3.78029
[1]	validation_0-rmse:3.77925
[2]	validation_0-rmse:3.77821
[3]	validation_0-rmse:3.77716
[4]	validation_0-rmse:3.77612
[5]	validation_0-rmse:3.77508
[6]	validation_0-rmse:3.77403
[7]	validation_0-rmse:3.77299
[8]	validation_0-rmse:3.77195
[9]	validation_0-rmse:3.77091
[10]	validation_0-rmse:3.76987
[11]	validation_0-rmse:3.76883
[12]	validation_0-rmse:3.76779
[13]	validation_0-rmse:3.76675
[14]	validation_0-rmse:3.76571
[15]	validation_0-rmse:3.76467
[16]	validation_0-rmse:3.76363
[17]	validation_0-rmse:3.76259
[18]	validation_0-rmse:3.76156
[19]	validation_0-rmse:3.76052
[20]	validation_0-rmse:3.75948
[21]	validation_0-rmse:3.75844
[22]	validation_0-rmse:3.75741
[23]	validation_0-rmse:3.75637
[24]	validation_0-rmse:3.75533
[25]	validation_0-rmse:3.75430
[26]	validation_0-rmse:3.75326
[27]	validation_0-rmse:3.75223
[28]	validation_0-rmse:3.75119
[29]	validation_0-rmse:3.75016
[30]	validation_0-rmse:3.74912
[31]	validation_0-rmse:3.74809
[32]	validation_0-

NameError: name 'clf' is not defined